In [11]:
from sklearn.linear_model import SGDClassifier
import numpy as np
import collections
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
import geometry
import itertools
from collections import defaultdict
import collections
import target
import controler
import utils
from functools import reduce
import linear_regression


class Task:

    def __init__(self, n, n_features, n_classes, class_sep, percent_flip, attack_proba, epsilon, seed = 42):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
        self.attack_proba = attack_proba

        self.n_features, self.n_classes = n_features, n_classes
        X, y = make_classification(n_samples = n, n_features = n_features,  n_informative = 2, n_redundant = 1, n_classes = n_classes, flip_y = percent_flip, class_sep = class_sep, n_clusters_per_class = 1 )
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=seed)
        self.X_train, self.y_train = X_train, y_train
        train_loader = DataLoader( TensorDataset( Tensor(X_train).to(self.device), Tensor(y_train).type(torch.LongTensor).to(self.device) ) , batch_size = 32, shuffle=True)
        test_loader = DataLoader(  TensorDataset( Tensor(X_test).to(self.device), Tensor(y_test).type(torch.LongTensor).to(self.device) ), batch_size = 1, shuffle=False)

        ### Train the target classifier:
        self.target = target.Target(n_classes,n_features)
        self.target.train(train_loader, 10)

        #attack_label = generate_sequence(self.target, 10, 10, test_loader, n_classes, epsilon)
        attack_label = [  np.random.choice([1,0], p=[attack_proba, 1-attack_proba]) for _ in range(len(X_test)) ]
        X_test = [ utils.fgsm_attack(self.target, epsilon, data[0], data[1], n_classes) if attack_label[i] else data[0] for i,data in enumerate(test_loader) ]

        self.test_loader = DataLoader(  TensorDataset( torch.stack( X_test ).to(self.device), Tensor(y_test).type(torch.LongTensor).to(self.device) ), batch_size = 1, shuffle=False)
        self.attack_label = torch.Tensor(attack_label).to(torch.float32)

                    
    # def offline(self):

    #     attack_clf = Controler( 2, self.n_features)
    #     #train_attack_label = generate_sequence(self.target, 10, 10, test_loader, n_classes, epsilon)
    #     train_attack_label = [  np.random.choice([1,0], p=[self.attack_proba, 1-self.attack_proba]) for _ in range(len(self.X_train)) ]
    #     train_loader = DataLoader( TensorDataset( Tensor(self.X_train).to(self.device), Tensor(self.train_attack_label).type(torch.LongTensor).to(self.device) ) , batch_size = 1, shuffle=True)
    #     X_train_attacks = [ fgsm_attack(self.target, epsilon, data[0], data[1], n_classes) if train_attack_label[i] else data[0] for i,data in enumerate(train_loader) ]
    #     train_loader = DataLoader( TensorDataset( Tensor(X_train_attacks).to(self.device), Tensor(self.y_train).type(torch.LongTensor).to(self.device) ) , batch_size = 1, shuffle=True)
        
    #     attack_clf.train(  train_loader, n_epochs=30)  
    #     y_pred = attack_clf.predict( self.X_test )

    #     tn, fp, fn, tp = confusion_matrix( self.attack_label , y_pred).ravel()
    #     print('True positive:{}, False positive:{}, Attacks:{}'.format(tp,fp,sum(self.attack_label) ) )

    def online(self, method, beta=1 ):

        control = controler.Controler( 2, self.n_features+self.n_classes)
        regrets = []
        K, attacks,i, abs_false_pos, abs_true_pos, false_pos, true_pos,  = 0,0,0,0,0,0,0
        for i,data in enumerate(self.test_loader):

            x, y_true = data[0][0], data[1] 
            is_attacked = self.attack_label[i]

            target_pred = self.target.clf(x) 
            controler_input = torch.concat( (x, target_pred), axis = 1)

            if method == 'STAP':
                initial_action, decision = utils.STAP(control.clf, controler_input, K, i) 
            elif method == 'Cesa':
                initial_action, decision = utils.CesaBianchi(control.clf, controler_input, beta, K)

            if decision == 1:
                if is_attacked != initial_action:
                    control.collect_fit([controler_input.detach()], is_attacked)
                    K = K+1  

            abs_true_pos = abs_true_pos+1 if decision == 1 and is_attacked == 1 else abs_true_pos
            abs_false_pos = abs_false_pos+1 if decision == 1 and is_attacked == 0 else abs_false_pos
            true_pos = true_pos+1 if initial_action == 1 and is_attacked == 1 else true_pos
            false_pos = false_pos+1 if initial_action == 1 and is_attacked == 0 else false_pos
            regret = abs( decision - is_attacked)

            regrets.append(regret.item())
            attacks = attacks+1 if is_attacked == 1 else attacks
            i+=1

        plt.plot( utils.cumsum(regrets) )
        plt.xlabel('Iteration')
        plt.ylabel('Cumulative Regret')
        print('     True positive:{},      False positive:{}'.format(true_pos,false_pos,attacks))
        print('Abs. True positive:{}, Abs. False positive:{}, Attacks:{}'.format(abs_true_pos,abs_false_pos,attacks))

    def f(self, alpha, t):
        return (alpha ** 1/3) * (t**2/3) * ( np.log(t)**1/3 )

    def W_k(self, L, H, pair,mathcal_K_plus):
        v_ij = geometry.observer_vector(L, H, pair[0], pair[1], mathcal_K_plus)[1]
        # print('v_ij',v_ij)
        res =  max( [ np.linalg.norm( v_ij[k] ) for k in mathcal_K_plus ]  )
        # print(res)
        return res

    # def get_confidence(x_t, t):
    #     d = 1 # what is the value of d?
    #     res = d * np.sqrt( d * np.log(t) + 2 * np.log( 1/ d_t)  )  * np.norm( x_t )
    #     return 

    def cpb_side(self,):
        
        alpha = 1.1 #alpha > 1
        L = np.array([ [0,1],[1,0] ])
        H = np.array([ ['no_feedback','no_feedback'], [0,1] ], dtype=object)
        mathcal_N = [0,1]
        mathcal_M = [0,1, 'no_feedback']
        N = len(L)
        n = np.zeros(N)
        mathcal_P = [ a for a in mathcal_N if geometry.isParetoOptimal(1, L)] # set of pareto optimal actions
        mathcal_K = [ pair for pair in list( itertools.combinations([0,1], 2) ) if geometry.areNeighbours(pair[0], pair[1], L) ] #set of unordered neighboring actions
        r = 0
        models = [ linear_regression.Model( self.n_features+self.n_classes ) for _ in range(N) ] 

        for t, data in enumerate(self.test_loader):

            x, y_true = data[0][0], data[1] 
            target_pred = self.target.clf(x) 
            model_input = torch.concat( (x, target_pred), axis = 1)
            #print(x, target_pred, y_true, model_input)
           
            if t < N:  # initialisation
                I  = t
                n[t] += 1
                J = H[ I ][ y_true ]
                # print( J)
                models[I].collect_fit([model_input.detach()], J)
                n[I] += 1
            else: 
                break
                
        for t,data in enumerate(self.test_loader):
            x, y_true = data[0][0], data[1] 
            target_pred = self.target.clf(x) 
            model_input = torch.concat( (x, target_pred), axis = 1)
            half_space = collections.defaultdict(dict)

            if t >= N:

                q, w = np.zeros(N), np.zeros(N)
                # print(mathcal_N)
                for i in mathcal_N:
                    q[i] = models[i].clf(model_input)  
                    w[i] =  models[i].confidence()

                # print('Q',q)
                # print('w', w)

                for pair in mathcal_K:
    
                    mathcal_K_plus =  geometry.Neighbourhood(pair[0], pair[1], L) #neighborhood action set of pair 
                    # print('mathcal_K_plus',mathcal_K_plus)
                    v_ij = geometry.observer_vector(L, H, pair[0], pair[1], mathcal_K_plus)
                    # print(v_ij)
                    # for k in mathcal_K_plus:
                    #     print(q[k])
                    #     print(v_ij[k])


                    d_ij = sum( [  v_ij[1][k].T * q[k]   for k in mathcal_K_plus ] )
                    c_ij = sum( [  np.linalg.norm( v_ij[1][k] ) * w[k]   for k in mathcal_K_plus ] )

                    print('prediction', d_ij)
                    if abs( d_ij ) >= c_ij:
                        half_space[ pair[0] ][ pair[1] ] = np.sign(d_ij)
                    else:
                        half_space[ pair[0] ][ pair[1] ] = 0
                    # print('halfspace',half_space)
                    # print('mathcal_P',mathcal_P)
                    # print('mathcal_K',mathcal_K)
                    mathcal_P_t, mathcal_K_t = geometry.get_polytope(half_space, L, mathcal_P, mathcal_K)

                mu_k = 0.1
                # print('mathcal_P_t, mathcal_K_t', mathcal_P_t, mathcal_K_t)
                K_plus_t = reduce(np.union1d, [ geometry.Neighbourhood(pair[0], pair[1], L) for pair in mathcal_K_t ]  )
                
                mathcal_V_t = K_plus_t # because in our problem instance all pairs are observable
                # print('mathcal_V_t',mathcal_V_t)
                mathcal_R_t = np.array( [ k for k in mathcal_N if n[k]<= mu_k * self.f(alpha, t) ] )
                # print('mathcal_R_t',mathcal_R_t)
                mathcal_S_t = reduce(np.union1d, [ mathcal_P_t , K_plus_t , np.intersect1d(mathcal_V_t, mathcal_R_t) ]  )
                mathcal_S_t = np.array(mathcal_S_t,dtype = np.int)
                # print('mathcal_S_t',mathcal_S_t)
                # print('n',n)
                values = [ self.W_k(L, H, pair,mathcal_K_plus)/n[i] for i in mathcal_S_t ]
                print('values', values)
                I = np.argmax(values)
                if I in np.setdiff1d( mathcal_V_t, np.concatenate([mathcal_P_t,K_plus_t]) ):
                    r += 1
                J = H[ I ][ y_true ]
                models[I].collect_fit([model_input.detach()], J)

    def f():
        return

    def exponential_weights(self, lbd, epsilon, L, H):

        L = np.array([ [0,1],[1,0] ])
        H = np.array([ ['no_feedback','no_feedback'], [0,1] ], dtype=object)
        
        mathcal_N = [0,1]
        mathcal_M = [0,1, 'no_feedback']
        mathcal_P = [ a for a in mathcal_N if geometry.isParetoOptimal(1, L)] # set of pareto optimal actions
        N = len(L)

        models = [ linear_regression.Model( self.n_features+self.n_classes ) for _ in range(N) ] 

        for t, data in enumerate(self.test_loader):

            x, y_true = data[0][0], data[1] 
            target_pred = self.target.clf(x) 
            model_input = torch.concat( (x, target_pred), axis = 1)

            Q = [   / sum( [  ] ) if a in mathcal_P else 0 for a in mathcal_N ] 

            np.random.choice([0,1,2], p=[0.9, 0.05, 0.05])

            y_t = f( A_t, sigma_t ) / P_t * A_t
        

    

In [12]:
n = 1000
n_classes = 2
classes =  np.arange(0, n_classes)
attack_proba =  0.1
epsilon = 10
n_features = 10
class_sep = 3
percent_flip = 0.1

task = Task(n, n_features, n_classes, class_sep, percent_flip, attack_proba, epsilon)

#task.cpb_side()
# task.online('STAP')
# task.online('Cesa', 3)
# print()
#print("Offline:")
#task.offline()

/home/mheuillet/Desktop/attack-detection/geometry.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x, res, rank, s = np.linalg.lstsq(A.T, Lij)
/tmp/ipykernel_31286/4132657865.py:203: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mathcal_S_t = np.array(mathcal_S_t,dtype = np.int)


prediction 2.8216095566749573
values [0.5, 0.5]
prediction -2.6624110341072083
values [0.5, 0.5]
prediction 2.819815158843994
values [0.5, 0.5]
prediction -2.9643797278404236
values [0.5, 0.5]
prediction 2.8768720626831055
values [0.5, 0.5]
prediction 3.7862765789031982
values [0.5, 0.5]
prediction 2.151483476161957
values [0.5, 0.5]
prediction 0.7608324587345123
values [0.5, 0.5]
prediction 3.2872979640960693
values [0.5, 0.5]
prediction 3.8174526691436768
values [0.5, 0.5]
prediction -1.2165143489837646
values [0.5, 0.5]
prediction -2.7129403948783875
values [0.5, 0.5]
prediction 2.6549551635980606
values [0.5, 0.5]
prediction -3.593640685081482
values [0.5, 0.5]
prediction 4.136091709136963
values [0.5, 0.5]
prediction 3.252321481704712
values [0.5, 0.5]
prediction 15.110772132873535
values [0.5, 0.5]
prediction -4.141800880432129
values [0.5, 0.5]
prediction 3.647374153137207
values [0.5, 0.5]
prediction 4.265752553939819
values [0.5, 0.5]
prediction -2.3239821791648865
values [0.5

In [7]:
class PMGame:

    def __init__(self, L, H):

        self.LossMatrix = L
        self.FeedbackMatrix = H
        self.N = L.shape[0] # Number of learner actions
        self.M = H.shape[0] # Number of environment outcomes
        self.Actions_dict = { a : "{0}".format(a) for a in range(self.N)} # Actions semantic
        self.Outcomes_dict = { a : "{0}".format(a) for a in range(self.M)} # Outcomes semantic





L = np.array([ [0,1],[1,0] ])
H = np.array([ ['no_feedback','no_feedback'], [0,1] ], dtype=object)

pm = PMGame(L,H)
print( geometry.ProblemClass(pm) )

print("Loss matrix:")
print(L)
for i in range(L.shape[0]):
    print()
    print("Domination matrix for action", i, ":" )
    print(geometry.domination_matrix(i,L) )
    print("Domination polytope:", geometry.DominationPolytope(i,L).minimized_generators() )
    print("dominating:", geometry.isNonDominated(i,L) )
    print("Strict Domination polytope:", geometry.StrictDominationPolytope(i,L).minimized_generators() )
    print("strictly dominating:", geometry.isStrictlyNonDominated(i,L) )
    print("degenerated:", geometry.isDegenerated(i,L) )
    print("Pareto Optimal:", geometry.isParetoOptimal(i, L) )



('easy', 'all neighbouring pairs are observable.')
Loss matrix:
[[0 1]
 [1 0]]

Domination matrix for action 0 :
[[ 0  0]
 [-1  1]]
Domination polytope: Generator_System {point(1/1, 0/1), point(1/2, 1/2)}
dominating: True
Strict Domination polytope: Generator_System {point(1/1, 0/1), closure_point(1/2, 1/2)}
strictly dominating: True
degenerated: False
Pareto Optimal: True

Domination matrix for action 1 :
[[ 1 -1]
 [ 0  0]]
Domination polytope: Generator_System {point(1/2, 1/2), point(0/1, 1/1)}
dominating: True
Strict Domination polytope: Generator_System {point(0/1, 1/1), closure_point(1/2, 1/2)}
strictly dominating: True
degenerated: False
Pareto Optimal: True
